# "Tensorflow Tutorials and Links"
> "Tf Tutorials"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [tensorflow, deep learning]
- image: images/some_folder/your_image.png
- hide: false


In [ ]:
# Dataset samples/examples
# Early stopping
# Tensorboard
# Saving checkpoint
# Saving Model
# Keras Tuner

Quickstart for Experts:<br>
<a href="https://www.tensorflow.org/tutorials/quickstart/advanced">Link</a>

# Convert to One hot encoded categorical datatype

In [ ]:
from keras.utils import to_categorical
y_train_b = to_categorical(y_train)

## Plotting sample Images from the Dataset

<a href="https://github.com/Sharan-Babu/Tf-reference/blob/master/Quickstart_for_beginners_tf.ipynb">
Link to colab notebook with MNIST DNN model building and visualization</a>

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap= plt.cm.binary)
    
    # u can map the index to class name using class_names[train_labels[i]] if needed
    plt.xlabel(class_names[y_train[i]])
plt.show()    

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

## Evaluating accuracy on test data

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

## Predictions on validation_data or batched data

In [ ]:
predictions = probability_model.predict(test_images)

## Prediction on a Single Image

In [ ]:
# Consider a single image from the dataset itself
img = test_images[1]

img = (np.expand_dims(img,0))

predictions_single = model.predict(img)

# Get prediction index position
prediction_index = np.argmax(predictions_single[0])

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

## TFDS and TF HUB for NLP Example

<a href="https://www.tensorflow.org/tutorials/keras/text_classification_with_hub">
    Link to the tutorial of the same.</a>

 <hr style="height:2px;border-width:0;color:blue;background-color:gray">

# EARLY STOPPING
### Use an EarlyStopping callback that tests a training condition for every epoch. If a set amount of epochs elapses without showing improvement, then automatically stop the training.

In [ ]:
# Function to build the Sequential tf.keras model
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
# tf.keras.callbacks.EarlyStopping is also an option
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop])

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

## Using Tensorboard for Model metric Visualizations
<a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"> Tensorboard usage example in google colab</a>

In [ ]:
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir/name)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[tensorboard_callback])

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Open an embedded TensorBoard viewer
# sizes is the name of the model
%tensorboard --logdir logs/fit

#%tensorboard --logdir {logdir}/sizes
#                               regularizers

## Example of L2 Regularization for preventing overfitting
<a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"> Link to notebook that shows code example</a>

In [ ]:
l2_model = tf.keras.Sequential([
    layers.Dense(512, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001),
                 input_shape=(FEATURES,)),
    layers.Dense(512, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1)])

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Saving checkpoints
<a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"> Link to notebook for saving models</a>

In [ ]:
# Required to save models in HDF5 format
!pip install -q pyyaml h5py

# Import necessary libraries
import os
import tensorflow as tf
from tensorflow import keras

# .......

model = create_model()

# Create checkpoint path - weights will be saved in a new training_1 folder
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
# If save_weights_only parameter is false then entire model is downloaded
# (assests, variables and .pb weights file)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1) 

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback]) 

# This creates a single collection of TensorFlow checkpoint files that are 
# updated at the end of each epoch
# ! ls {checkpoint_dir} -- To check the saved files in the directory from colab

Create a new, untrained model. When restoring a model from weights only, you must have a model with the same architecture as the original model. Since it's the same model architecture, you can share weights despite that it's a different instance of the model.

In [ ]:
# Create a basic model instance
model = create_model()
model.load_weights(checkpoint_path)
# Now the model is restored

##### <u>Saving checkpoint every 5 epochs and loading the latest checkpoint

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)

# Steps to Load this model checkpoints
# 1) Choose the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)

# 2) Create a new model instance
model.load_eights(latest)

# Model restored!

##### <u> Manually save weights

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

##### <u> Save the entire model

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

!ls saved_model/my_model
# O/p: assets   saved_model.pb   variables

# Restoring model
new_model = tf.keras.models.load_model('saved_model/my_model')

##### <u> Save in HDF5 format

In [ ]:
model.save('my_model.h5')

# Restore the model
new_model = tf.keras.models.load_model('my_model.h5')

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Keras Tuner:
<a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"> Link to notebook with code for auto tuner </a>
## Hyper Parameter Tuning

In [ ]:
# Necessary pre-steps apart from the usual ones
!pip install -q -U keras-tuner
import kerastuner as kt

# Ex:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
    return model


# Instantiate the tuner and perform hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

# To ensure training output is cleared after each trial thereby keeping 
# notebook clean and not cluttered
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"{best_hps.get('units')}")



# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Pre-processing if Dataset is in numpy format
<a href="https://www.tensorflow.org/tutorials/load_data/numpy">Link to tutorial</a>

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Pre-Processing Image Data
<a href="https://www.tensorflow.org/tutorials/load_data/images">Link to tutorial</a>

In [ ]:
# Ex:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

# Imp:
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

# After this code is same as usual.

 <hr style="height:2px;border-width:0;color:gray;background-color:pink">

# Pre-processing Text Data
<a href="https://www.tensorflow.org/tutorials/load_data/text"> Tutorial Link</a>

 <hr style="height:2px;border-width:0;color:gray;background-color:green">

# Unicode pre-processing
<a href="https://www.tensorflow.org/tutorials/load_data/unicode"> Tutorial</a>

 <hr style="height:2px;border-width:0;color:gray;background-color:cyan">

# Custom Layer Building
<a href="https://www.tensorflow.org/tutorials/customization/custom_layers">Tutorial</a><br><br>
<a href="https://github.com/aymericdamien/TensorFlow-Examples/blob/master/tensorflow_v2/notebooks/4_Utils/build_custom_layers.ipynb">Tutorial 2 </a>
 <hr style="height:2px;border-width:0;color:gray;background-color:gray">
# Custom Training
<a href="https://www.tensorflow.org/tutorials/customization/custom_training">Tutorial</a>

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Image Classification with augmentation (as layers)
<a href="https://www.tensorflow.org/tutorials/images/classification">Link</a>

In [ ]:
!pip install -q tf-nightly

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

# Download and explore the dataset
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

# Make a list of the returned generator function which gets back only.jpg images
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

# List of all images of roses. Make it str to read it.
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

# Loading using keras.preprocessing
batch_size = 32
img_height = 180
img_width = 180

#.image_dataset_from_directory currently only in tf-nightly
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

# create model
num_classes = 5

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# compile model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train model
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# Data augmentation
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

# Visualizing augmentation
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")
    
# adding augmentation and dropout layers
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# Compile,train and fit......

# Predict on new data
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

 <hr style="height:2px;border-width:0;color:gray;background-color:cyan">


# Transfer Learning With TF HUB
<a href="https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub">Link</a>

In [ ]:
# Setup
!pip install -q -U tf-hub-nightly
!pip install -q tfds-nightly
import tensorflow_hub as hub

# ImageNet classifier
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" 
IMAGE_SHAPE = (224, 224)

# Make it a Keras layer
classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

# Labels- Now u can directly use this to predict on classes this was built
# to predict or use it for ur own dataset
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

# Transfer learning for custom dataset
# Download the headless model
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" 

feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))
feature_extractor_layer.trainable = False

# Attach a classification head
model = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(image_data.num_classes)
])

model.summary()

# Now compile and fit.....

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Transfer Learning with Pre-Trained CNN

In [ ]:
# pre trained model for CATS VS DOGS
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

IMG_SIZE = (160, 160)

# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

# This feature extractor converts each 160x160x3 image into a 5x5x1280 block of features
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

# DATA AUGMENTATION LAYERS
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# Freeze the convolutional base
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

# compile model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# train model
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

# Fine Tuning
base_model.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

# Now u can compile and fit again    
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Object Detection
<a href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb#scrollTo=vchaUW1XDodD">Link to Colab Notebook</a>

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Classification on Imbalanced Data - DL

<a href="https://www.tensorflow.org/tutorials/structured_data/imbalanced_data"> Link to tutorial</a>

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">


# Time Series Forcasting

<a href="https://www.tensorflow.org/tutorials/structured_data/time_series">Link</a>

 <hr style="height:2px;border-width:0;color:gray;background-color:gray">
